In [114]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import numpy as np

In [115]:
reviews = pd.read_csv('C:/JK/Masters Studies/Spring 2018/Information Retrieval/Project/data/reviews.csv/reviews.csv')
# reviews['listing_id'].value_counts()
# vc = reviews['listing_id'].value_counts()

In [116]:
reviews_less = reviews.groupby('listing_id').filter(lambda g: (g.listing_id.size >= 5) and (g.listing_id.size < 15))

In [117]:
def remove_nonalphanumeric(text):
    return re.sub("[^a-zA-Z0-9]"," ", str(text)).lower()

In [ ]:
unique_listings = reviews.listing_id.unique()
len(unique_listings)

3986

In [ ]:
reviews['comments_1'] = reviews['comments'].apply(remove_nonalphanumeric)
reviews['tokenized_comments'] = reviews['comments_1'].apply(word_tokenize)
stopset = stopwords.words('english') + list(string.punctuation)
reviews['stop_comments'] = reviews['tokenized_comments'].apply(lambda x: [item for item in x if item not in stopset])
def func(row):
    return " ".join(row)
reviews['data'] = reviews['stop_comments'].apply(lambda x: func(x))

In [ ]:
def display_topics(H,W,feature_names,documents,documents_id,listing_id,words,no_top_words,no_top_documents):
    result_list = []
    for topic_idx, topic in enumerate(H):
        message = " ".join(([feature_names[i]
                             for i in topic.argsort()[:-no_top_words - 1:-1]]))
        top_doc_indices = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            result = (message,documents_id[doc_index],listing_id[doc_index],documents[doc_index],words[doc_index])
            result_list.append(result)

    return result_list

In [ ]:
no_top_words = 3
no_top_documents = 5
result=[]

In [ ]:
reviews=reviews.groupby('listing_id').filter(lambda g: (g.listing_id.size >= 100))

unique_listings = reviews.listing_id.unique()
len(unique_listings)

In [ ]:
for each_listing in unique_listings:
#     print(each_listing)
    
    listing_reviews=reviews[reviews['listing_id']==each_listing]
    tf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=2, max_features=10000)
    tf = tf_vectorizer.fit_transform(listing_reviews['data'])
    tf_feature_names = tf_vectorizer.get_feature_names()
    nmf_model = NMF(n_components= 3, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd')
    nmf=nmf_model.fit(tf)
    nmf_W = nmf.transform(tf)
    nmf_H = nmf.components_
    output = display_topics(nmf_H,nmf_W,tf_feature_names,list(listing_reviews['comments']),list(listing_reviews['id']),list(listing_reviews['listing_id']),list(listing_reviews['data']),no_top_words,no_top_documents)
    result += output
#     print(output)
# np.save('nmf_H.npy', nmf_H)
# np.save('nmf_tf.npy',tf)

result_df = pd.DataFrame(result, columns=['Topic','Review_Id','Listing_Id','Documents','Words'])
result_df.to_csv("Result_nmf_100.csv",columns=["Topic","Review_Id","Listing_Id","Documents","Words"])